In [27]:
import numpy as np
import pandas as pd
import random
import torch
import os
seed = 42
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)

In [28]:
from datasets import load_dataset

In [29]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("friedrichor/MSR-VTT", "train_9k")['train']

In [30]:
print(ds)

Dataset({
    features: ['video_id', 'video', 'caption', 'source', 'category', 'url', 'start time', 'end time', 'id'],
    num_rows: 9000
})


In [31]:
print(ds.column_names)
print(ds.features)

['video_id', 'video', 'caption', 'source', 'category', 'url', 'start time', 'end time', 'id']
{'video_id': Value('string'), 'video': Value('string'), 'caption': List(Value('string')), 'source': Value('string'), 'category': Value('int64'), 'url': Value('string'), 'start time': Value('float64'), 'end time': Value('float64'), 'id': Value('int64')}


In [32]:
sampled_ds = ds.shuffle(seed=seed).select(range(100))
print(sampled_ds)

Dataset({
    features: ['video_id', 'video', 'caption', 'source', 'category', 'url', 'start time', 'end time', 'id'],
    num_rows: 100
})


In [33]:
sample = sampled_ds[0]
print(sample["video_id"])
print(sample["caption"][:2])   # first two captions
print(sample["url"])
print(sample["start time"], sample["end time"])


video2015
['a class full of women move their body watching a black guy dance', 'a dance instructor showing a class how to dance certain moves']
https://www.youtube.com/watch?v=CHeSIvrCdjc
52.68 65.25


In [34]:
data_path = "data/MSR-VTT"

In [35]:
os.makedirs(data_path, exist_ok=True)

In [42]:
import subprocess
failed_count = 0
def download_clip(clip_info):
    # Download and trim using ffmpeg
    url =  clip_info['url']
    start = clip_info['start time']
    end = clip_info['end time']
    out = os.path.join(data_path, f"{clip_info['video_id']}.mp4")
    global failed_count
    failed_count += 1
    if os.path.exists(out):
        print(f"File {out} already exists, skipping download.")
        return out
    try:
        cmd = [
            "yt-dlp", "-f", "bestvideo*+bestaudio/best",
            "--download-sections", f"*{start}-{end}",
            "-o", out,
            url
        ]
        subprocess.run(cmd, check=True)
        return out
    except Exception as e:
        print(f"Failed {clip_info['video_id']}: {e}")
        return ""
sampled_ds = sampled_ds.map(
    lambda x: {"clip_path": download_clip(x)},
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Failed video4308: Command '['yt-dlp', '-f', 'bestvideo*+bestaudio/best', '--download-sections', '*82.0-94.73', '-o', 'data/MSR-VTT\\video4308.mp4', 'https://www.youtube.com/watch?v=gmqgVxIkehw']' returned non-zero exit status 1.
Failed video1839: Command '['yt-dlp', '-f', 'bestvideo*+bestaudio/best', '--download-sections', '*50.23-64.58', '-o', 'data/MSR-VTT\\video1839.mp4', 'https://www.youtube.com/watch?v=MoEGd7B-OA0']' returned non-zero exit status 1.
Failed video102: Command '['yt-dlp', '-f', 'bestvideo*+bestaudio/best', '--download-sections', '*45.76-58.06', '-o', 'data/MSR-VTT\\video102.mp4', 'https://www.youtube.com/watch?v=ZayRbTmhYTc']' returned non-zero exit status 1.
Failed video9315: Command '['yt-dlp', '-f', 'bestvideo*+bestaudio/best', '--download-sections', '*128.97-139.94', '-o', 'data/MSR-VTT\\video9315.mp4', 'https://www.youtube.com/watch?v=3BD2F0QUS5I']' returned non-zero exit status 1.
Failed video4654: Command '['yt-dlp', '-f', 'bestvideo*+bestaudio/best', '--downl

In [44]:
missing_ds = sampled_ds.filter(lambda x: not x.get("clip_path"))
print(missing_ds)

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['video_id', 'video', 'caption', 'source', 'category', 'url', 'start time', 'end time', 'id', 'clip_path'],
    num_rows: 36
})


In [47]:
sampled_ds = sampled_ds.filter(
    lambda x: x.get("clip_path") not in ("", None)
)
print("Rows after drop:", len(sampled_ds))

Filter:   0%|          | 0/64 [00:00<?, ? examples/s]

Rows after drop: 64


In [48]:
sampled_ds.to_csv("data/sampled_msrvtt_clips.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

71419